In [37]:
from bs4 import BeautifulSoup
import requests
import re
import requests
import pandas as pd

def search_company_page(company):
    #Given a company, find its webpage URL through the research system of Reuter

    #This URL return the search page for the given company
    search_url = f'https://www.reuters.com/search/news?sortBy=&dateRange=&blob={company}'

    #Creating url request and bs4 object
    req=requests.get(search_url)

    soup = BeautifulSoup(req.content, "html.parser")
    company_page = soup.find("div",class_="search-stock-ticker").findNext()["href"]
        
    #we return the final url of the webpage of the company
    return re.sub(r'overview','financial-highlights',"https://www.reuters.com"+company_page)

def get_data(company):
    url=search_company_page(company)
    req=requests.get(url)
    soup = BeautifulSoup(req.content, "html.parser")

    ##Get stock price from company page
    try:
        price=float(soup.find("div",class_="sectionQuote nasdaqChange").
                  findAll("span")[1].
                  contents[0].
                  strip()
                 )
    except:
        price = "?"

    ##Get %change from company page
    try:
        change = float(re.sub(r'^[(]|[%)]+$',"",
                          soup.
                          find("div",class_="sectionQuote priceChange").
                          findAll("span")[4].
                          contents[0].
                          strip()))/100
    except:
        change = re.sub(r'^[(]|[%)]+$',"",
                          soup.
                          find("div",class_="sectionQuote priceChange").
                          findAll("span")[4].
                          contents[0].
                          strip())

    try:
    ##Get Shares owned by institutionnal holders
        shared_owned, =(float(re.sub(r'^[(]|[%)]+$',"",elmt.parent.parent.find(class_="data").contents[0]))
                        for elmt in soup.findAll("strong") 
                        if "Shares Owned" in elmt.contents[0])
    except:
        shared_owned = "?"
    

    ##Get divident yield for company/industry/sector
    try:
        company_divident_yield, *rest = (float(elmt.parent.findAll(class_="data")[0].get_text())
                                   for elmt in soup.findAll("td") 
                                   if "Dividend Yield" in elmt.get_text())
    except:
        company_divident_yield = "?"
        
    try:    
        industry_divident_yield, *rest = (float(elmt.parent.findAll(class_="data")[1].get_text())
                                   for elmt in soup.findAll("td") 
                                   if "Dividend Yield" in elmt.get_text())
    except:
        industry_divident_yield = "?"

    try:    
        sector_divident_yield, *rest = (float(elmt.parent.findAll(class_="data")[2].get_text())
                                   for elmt in soup.findAll("td") 
                                   if "Dividend Yield" in elmt.get_text())
    except:
        sector_divident_yield = "?"
    
    try:
        sell_year_ending, *rest = (int(elmt.parent.findAll(class_="data")[0].get_text())
                                   for elmt in soup.findAll("td") 
                                   if "Quarter Ending" in elmt.get_text())
    except:
        sell_year_ending="?"
        
    try:
        market = soup.find("span",class_="nasdaqChangeHeader").text.split("on ")[1]
    except:
        marker = ""

    return {"company":company.upper(),
            "price":price,
            "change":change,
            "shared_owned":shared_owned,
            "company_divident_yield":company_divident_yield,
            "industry_divident_yield":industry_divident_yield,
            "sell_year_ending":sell_year_ending}
        

In [40]:
reuter_L = []
company_list = ["Airbus","Danone","LVMH","total","edf","michelin","carrefour",'valeo','renault','peugeot','technipfmc']
market="Paris"
for company in company_list:
    reuter_L.append(get_data(company))

reuter_df = pd.DataFrame(reuter_L)

In [41]:
reuter_df

,change,company,company_divident_yield,industry_divident_yield,price,sell_year_ending,shared_owned
0,-0.0342,AIRBUS,1.45,1.34,98.99,5,43.53
1,-0.0047,DANONE,2.49,2.35,65.33,5,50.60
2,-0.0731,LVMH,1.92,1.70,265.15,3,20.57
3,-0.0552,TOTAL,9.29,3.32,173.00,?,0.84
4,-0.0151,EDF,?,3.90,12.51,?,16.62
5,-0.0414,MICHELIN,3.43,2.40,99.26,?,39.16
6,0.0196,CARREFOUR,2.94,1.57,15.63,1,23.40
7,-0.0293,VALEO,3.9,1.87,31.50,1,54.67
8,-0.0066,RENAULT,4.94,2.88,71.12,?,29.42
9,-0.0476,PEUGEOT,2.44,2.88,20.62,1,36.40


In [17]:
search_company_page("technip")

<tr>
<th>Company</th>
<th>Symbol</th>
<th>Exchange</th>
</tr>

In [47]:
company="technip"
url=search_company_page(company)
req=requests.get(url)
soup = BeautifulSoup(req.content, "html.parser")
soup.find("span",class_="nasdaqChangeHeader").text.split("on ")[1]

'New York Stock Exchange'